# Wrong gradient

From OSCIGRAD
When the function values and gradient contain values that are far apart, the GP will have large errors in the prediction.
Best thing to do seems to be to alter kernel hyperparameter. Most accurate kernel does not align with the one with max log marginal likelihood
Linear scaling does not help, since distance remains in same order and thus error decreases only in the absolute but not relative.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import gaussian_process.GPfunctions as gp
from gaussian_process import GaussianProcess
from gaussian_process.kernels import SquaredExponentialKernel, CubicKernel
import os

In [ ]:
X = np.linspace(start=0.0, stop=1.0, num=1_000)

In [ ]:
X_train = [
    0.00000000e00,
    1.00000000e00,
    2.01370962e-01,
    2.85678888e-10,
    3.12057312e-09,
    3.12057212e-09,
]
f_train = [
    4.88153517e-12,
    1.41555445e08,
    5.60060592e06,
    9.24588434e-12,
    1.28168841e-09,
    1.28168757e-09,
]
g_train = [
    -2.44151587e-02,
    2.93727546e08,
    5.55308816e07,
    5.49693870e-02,
    8.42730769e-01,
    8.42730492e-01,
]
f_noise = 1e-10
g_noise = 1e-10

# Scaling
# factor = 1e-8
# f_train = [factor * f for f in f_train]
# g_train = [factor * g for g in g_train]

kernel = SquaredExponentialKernel(1.0 / (len(X_train) - 1))  # Average distance
# kernel = SquaredExponentialKernel(min([np.abs(a-b) for a,b in itertools.pairwise(sorted(X_train))])) # Min distance
# kernel = SquaredExponentialKernel(1e-5) # Max log-marginal-likelihood

GP_posterior = GaussianProcess(
    kernel,
    x_known=X_train,
    f_known=f_train,
    g_known=g_train,
    f_noise=f_noise,
    g_noise=g_noise,
)

posterior_mean, posterior_variance = GP_posterior(X)
posterior_std = GP_posterior.std_deviation(X, variance=posterior_variance)
posterior_mean_derivative, posterior_mean_derivative_variance = GP_posterior.derivative(
    X
)

fig, (ax1) = plt.subplots(1, 1, sharey=True)

ax1.scatter(X_train, f_train, label="Observations")
gp.plot_gp(ax1, X, posterior_mean, posterior_std)
gp.plot_label(ax1, "Gaussian Process")

fig.suptitle("Squared exponential kernel sampled")
fig.set_figwidth(15)
plt.show()

print("For gp with squared exponential kernel and gradient info we have:")
for x, f, g in zip(X_train, f_train, g_train):
    m, v = GP_posterior(x)
    md, mdv = GP_posterior.derivative(x)
    print(
        f"x={x}:",
        f"  abs(f(x)-gp(x))={np.abs(f-m)} with f(x)={f} and gp(x)={m}",
        f"  abs(g(x)-g_gp(x))={np.abs(g-md)} with g(x)={g} and g_gp(x)={md}",
        sep=os.linesep,
    )
print(f"With total f-error of {GP_posterior.f_error()}")
print(f"With total g-error of {GP_posterior.g_error()}")